In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Importing libraries

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from scipy.special import expit
import pandas as pd

### Function for converting rgb images to grayscale

In [3]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

### Loading images and building flattened image dataset

Images have been downscaled to 80x80 resolution 

In [4]:
images = []
labels = []
files_list=[]
for _, _, files_list in os.walk("./dataset"):
    pass
for file in files_list:
    label, rest = file.split('_')
    gray = mpimg.imread("./dataset/"+file)
    gray = rgb2gray(gray)
    gray = cv2.resize(gray, (80,80), interpolation = cv2.INTER_AREA)
    images.append(gray.flatten())
    labels.append(label)
images = np.array(images)
images = np.hstack((np.ones((images.shape[0], 1)), images))
labels = np.array(labels)

### Train-validation split (80-20)

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(images, labels, train_size=.8)

### Normalizing feature values using minmax scalar

In [6]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)

### Sigmoid function

In [7]:
sigmoid = lambda x : (1 / (1 + np.exp(-x)))

### Logistic regression training (one vs all) (Images without PCA)

In [32]:
weights = np.ones((x_train.shape[1], 1))
mod_y_train = pd.get_dummies(y_train)
tol = 0.000001
alpha = 0.008
epsilon = 1e-7
m = x_train.shape[0]
for cind in range(len(set(y_train))):
    maj_class = mod_y_train.columns[cind]
    print("Training classifier for class ",maj_class)
    thetas = (np.random.rand(x_train.shape[1], 1) - .5)
    ly_train = np.array(mod_y_train.iloc[:,cind]).reshape((-1,1))
    error = 10*tol
    pcost = 0
    iters=0
    while error >= tol and iters <= 10000:
        hypo = np.dot(x_train, thetas)
        hsig = (sigmoid(hypo))
        y1cost = np.dot(ly_train.transpose(), np.log(hsig + epsilon))
        y0cost = np.dot((1 - ly_train).transpose(), np.log(1 - hsig + epsilon))
        cost = (-1/m)*( y1cost + y0cost )

        error = abs(pcost - cost)
        pcost = cost
        thetas -= (alpha/m)*np.dot(x_train.transpose(), (hsig - ly_train))
        iters += 1
    cost = cost[0][0]
    print("Iterations = ",iters," Cost = ",cost)
    weights = np.hstack((weights, thetas))
weights = weights[:,1:] 

Training classifier for class  000
Iterations =  10001  Cost =  0.039972783862149794
Training classifier for class  001
Iterations =  10001  Cost =  0.04258302413878414
Training classifier for class  002
Iterations =  10001  Cost =  0.026233038892598537
Training classifier for class  003
Iterations =  10001  Cost =  0.021071587010701956
Training classifier for class  004
Iterations =  10001  Cost =  0.030138330607002384
Training classifier for class  005
Iterations =  10001  Cost =  0.028845489506244437
Training classifier for class  006
Iterations =  10001  Cost =  0.05582650420112665
Training classifier for class  007
Iterations =  10001  Cost =  0.026607480207931695


### Validation dataset's classsifcation report

In [35]:
y_pred = []
res = sigmoid(np.dot(x_valid, weights))
# print(res)
for row in range(res.shape[0]):
    ind = np.argmax(res[row,:])
    y_pred.append(mod_y_train.columns[ind])
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

         000       0.75      0.75      0.75        12
         001       0.78      0.47      0.58        15
         002       0.69      0.79      0.73        14
         003       0.75      0.71      0.73        17
         004       0.44      0.78      0.56         9
         005       0.73      0.53      0.62        15
         006       0.62      0.62      0.62        13
         007       0.73      0.89      0.80         9

    accuracy                           0.67       104
   macro avg       0.68      0.69      0.67       104
weighted avg       0.70      0.67      0.67       104



## Comparison with Sklearn's logistic regressor

In [36]:
clf = LogisticRegression(random_state=0,  max_iter=10000).fit(x_train, y_train)
y_pred = clf.predict(x_valid)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

         000       0.62      0.83      0.71        12
         001       0.86      0.40      0.55        15
         002       0.59      0.71      0.65        14
         003       0.83      0.88      0.86        17
         004       0.50      0.78      0.61         9
         005       0.90      0.60      0.72        15
         006       0.60      0.46      0.52        13
         007       0.75      1.00      0.86         9

    accuracy                           0.69       104
   macro avg       0.71      0.71      0.68       104
weighted avg       0.72      0.69      0.68       104



# Logistic regression with PCA

## Calculating covariance marix

#### Function to calculate covariance matrix

In [11]:
def calc_cov_mat(ip_mat):
    mean_mat = np.sum(ip_mat, axis=0)/ip_mat.shape[0]
    mat_b = ip_mat - mean_mat
    return np.dot(mat_b.transpose(), mat_b)/(ip_mat.shape[0] - 1)

In [12]:
cov_mat = calc_cov_mat(x_train)
cov_mat.shape

(6401, 6401)

#### Applying eigendecomposition to covariance matrix

In [13]:
evals, evecs = np.linalg.eig(cov_mat)

#### Sorting in descending order of eigenvalues

In [14]:
idx = evals.argsort()[::-1]   
eigenValues = evals[idx]
eigenVectors = evecs[:,idx]

In [15]:
print(eigenValues)
print(eigenVectors)

[ 5.98814585e+01+0.00000000e+00j  4.39872596e+01+0.00000000e+00j
  2.62559375e+01+0.00000000e+00j ... -1.27882978e-15+0.00000000e+00j
 -1.48922216e-15+1.63824644e-16j -1.48922216e-15-1.63824644e-16j]
[[ 0.        +0.j          0.        +0.j          0.        +0.j
  ...  0.        +0.j          0.        +0.j
   0.        -0.j        ]
 [-0.00624637+0.j          0.01099566+0.j         -0.0140834 +0.j
  ...  0.01529667+0.j         -0.01637492+0.00997025j
  -0.01637492-0.00997025j]
 [-0.00646395+0.j          0.0113859 +0.j         -0.01217362+0.j
  ...  0.00572676+0.j         -0.00754652-0.01834948j
  -0.00754652+0.01834948j]
 ...
 [-0.0026727 +0.j         -0.01446198+0.j         -0.0305677 +0.j
  ...  0.01297333+0.j         -0.01847452+0.03301794j
  -0.01847452-0.03301794j]
 [-0.00196671+0.j         -0.01365385+0.j         -0.03148079+0.j
  ... -0.02586671+0.j          0.00366498-0.00194799j
   0.00366498+0.00194799j]
 [-0.0018722 +0.j         -0.01323929+0.j         -0.03229916+0.j
  

### Deciding no. of components to keep corresponding to 90% variance retention

In [37]:
target_var = .9
tot_var = np.sum(eigenValues)
perc = []
csum=0
comps = 0
tcomps=0
for ind in range(250):
    csum += eigenValues[ind]
    comps += 1
    perc.append(csum/tot_var)
    if np.real(csum/tot_var) >= target_var:
        print(target_var*100,"% variance achieved at ",comps," components")
        tcomps=comps
        break

90.0 % variance achieved at  69  components


#### Obtaining the PCA transformation (projection) and reconstruction matrix

In [17]:
trans_mat = np.real(eigenVectors[:,0:tcomps])
recon_mat = np.dot(trans_mat, trans_mat.transpose())

### Dimensionality reduction of all images

In [38]:
x_train_pca = np.dot(x_train, trans_mat)
x_valid_pca = np.dot(x_valid, trans_mat)

### Normalizing feature values using minmax scalar

In [39]:
scaler = MinMaxScaler()
x_train_pca = scaler.fit_transform(x_train_pca)
x_valid_pca = scaler.transform(x_valid_pca)

#### Classifier training

In [40]:
weights = np.ones((x_train_pca.shape[1], 1))
mod_y_train = pd.get_dummies(y_train)
tol = 0.000001
alpha = 0.05
epsilon = 1e-7
m = x_train_pca.shape[0]
for cind in range(len(set(y_train))):
    maj_class = mod_y_train.columns[cind]
    print("Training classifier for class ",maj_class)
    thetas = (np.random.rand(x_train_pca.shape[1], 1) - .5)
    ly_train = np.array(mod_y_train.iloc[:,cind]).reshape((-1,1))
    error = 10*tol
    pcost = 0
    iters=0
    while error >= tol and iters <= 10000:
        hypo = np.dot(x_train_pca, thetas)
        hsig = (sigmoid(hypo))
        y1cost = np.dot(ly_train.transpose(), np.log(hsig + epsilon))
        y0cost = np.dot((1 - ly_train).transpose(), np.log(1 - hsig + epsilon))
        cost = (-1/m)*( y1cost + y0cost )

        error = abs(pcost - cost)
        pcost = cost
        thetas -= (alpha/m)*np.dot(x_train_pca.transpose(), (hsig - ly_train))
        iters += 1
    cost = cost[0][0]
    print("Iterations = ",iters," Cost = ",cost)
    weights = np.hstack((weights, thetas))
weights = weights[:,1:] 

Training classifier for class  000
Iterations =  10001  Cost =  0.20296230413992
Training classifier for class  001
Iterations =  10001  Cost =  0.21092571882036149
Training classifier for class  002
Iterations =  10001  Cost =  0.16561587605998987
Training classifier for class  003
Iterations =  10001  Cost =  0.13454870469317848
Training classifier for class  004
Iterations =  10001  Cost =  0.16348693273381043
Training classifier for class  005
Iterations =  10001  Cost =  0.17154771972992633
Training classifier for class  006
Iterations =  10001  Cost =  0.2132857668529178
Training classifier for class  007
Iterations =  10001  Cost =  0.14801623124255484


### Validation dataset's classsifcation report

In [41]:
y_pred = []
res = sigmoid(np.dot(x_valid_pca, weights))
# print(res)
for row in range(res.shape[0]):
    ind = np.argmax(res[row,:])
    y_pred.append(mod_y_train.columns[ind])
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

         000       0.67      0.83      0.74        12
         001       0.80      0.53      0.64        15
         002       0.50      0.50      0.50        14
         003       0.92      0.71      0.80        17
         004       0.40      0.67      0.50         9
         005       0.83      0.67      0.74        15
         006       0.45      0.38      0.42        13
         007       0.64      1.00      0.78         9

    accuracy                           0.64       104
   macro avg       0.65      0.66      0.64       104
weighted avg       0.68      0.64      0.65       104



## Comparison with Sklearn's logistic regressor

In [42]:
clf = LogisticRegression( max_iter=10000).fit(x_train_pca, y_train)
y_pred = clf.predict(x_valid_pca)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

         000       0.59      0.83      0.69        12
         001       0.67      0.40      0.50        15
         002       0.57      0.57      0.57        14
         003       0.93      0.82      0.87        17
         004       0.33      0.56      0.42         9
         005       0.90      0.60      0.72        15
         006       0.50      0.46      0.48        13
         007       0.67      0.89      0.76         9

    accuracy                           0.63       104
   macro avg       0.64      0.64      0.63       104
weighted avg       0.67      0.63      0.64       104

